# Data

## Importing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import random

import seaborn as sns
import os
%matplotlib inline

In [2]:
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams.update({'font.size': 12})

## Reading Data

In [3]:
class StockDataSet(object):
    
    def __init__(self,
                 stock_sym="GSPC.csv",
                 input_size=1,
                 num_steps=30,
                 test_ratio=0.1,
                 normalized=True,
                 close_price_only=True):
        self.stock_sym = stock_sym
        self.input_size = input_size
        self.num_steps = num_steps
        self.test_ratio = test_ratio
        self.close_price_only = close_price_only
        self.normalized = normalized

        self.raw_df = self.read_dataset()

        # Merge into one sequence
        if close_price_only:
            self.raw_seq = self.raw_df['Close']
        else:
            self.raw_seq = [price for tup in self.raw_df[['Open', 'Close']].values for price in tup]

        self.raw_seq = np.array(self.raw_seq)
        self.train_X, self.train_y, self.test_X, self.test_y = self._prepare_data(self.raw_seq)

        
    def read_dataset(self):
        cwd = os.getcwd()
        dataset_path = os.path.join(cwd, self.stock_sym)
        data = pd.read_csv(dataset_path, parse_dates=['Date'], infer_datetime_format=True)
        return data

    
    def _prepare_data(self, seq):
        # split into items of input_size
        seq = [np.array(seq[i * self.input_size: (i + 1) * self.input_size])
               for i in range(len(seq) // self.input_size)]

        if self.normalized:
            seq = [seq[0] / seq[0][0] - 1.0] + [
                curr / seq[i][-1] - 1.0 for i, curr in enumerate(seq[1:])]

        # split into groups of num_steps
        X = np.array([seq[i: i + self.num_steps] for i in range(len(seq) - self.num_steps)])
        y = np.array([seq[i + self.num_steps] for i in range(len(seq) - self.num_steps)])

        train_size = int(len(X) * (1.0 - self.test_ratio))
        train_X, test_X = X[:train_size], X[train_size:]
        train_y, test_y = y[:train_size], y[train_size:]
        return train_X, train_y, test_X, test_y
    

    def generate_one_epoch(self, batch_size):
        num_batches = int(len(self.train_X)) // batch_size
        if batch_size * num_batches < len(self.train_X):
            num_batches += 1

        batch_indices = list(range(num_batches))
        random.shuffle(batch_indices)
        for j in batch_indices:
            batch_X = self.train_X[j * batch_size : (j + 1) * batch_size]
            batch_y = self.train_y[j * batch_size : (j + 1) * batch_size]
            assert set(map(len, batch_X)) == {self.num_steps}
            yield batch_X, batch_y

In [4]:
dataset = StockDataSet().raw_df

## About Data

In [ ]:
dataset.head()

In [ ]:
dataset.describe()

In [ ]:
dataset.shape

In [ ]:
time_n_closing_value = dataset.iloc[:,[0,4]]
time_n_closing_value['Date']=pd.to_datetime(time_n_closing_value['Date'], format='%d/%m/%Y')
time_n_closing_value.set_index(['Date'], inplace=True)

In [ ]:
time_n_closing_value.plot()
plt.title('Date Vs Close Price Graph')
plt.ylabel('Close Price')
plt.xlabel('Date')
plt.show()

# Tensorflow Implementation

## Configuration for RNN

In [ ]:
class RNNConfig():
    input_size=3
    num_steps=30
    lstm_size=128
    num_layers=1
    keep_prob=0.8
    batch_size = 64
    init_learning_rate = 0.005
    learning_rate_decay = 0.99
    init_epoch = 5
    max_epoch = 100

config = RNNConfig()

## LSTM Implementation

### Importing Modules

In [ ]:
import tensorflow as tf
import numpy as np

### Creating Graph

In [ ]:
def create_lstm_graph():
    
    tf.reset_default_graph()
    lstm_graph = tf.Graph()

    with lstm_graph.as_default():

        inputs = tf.placeholder(tf.float32, [None, config.num_steps, config.input_size], name="Inputs")
        targets = tf.placeholder(tf.float32, [None, config.input_size], name="Targets")
        learning_rate = tf.placeholder(tf.float32, None, name="learn_rate")

        def _create_one_cell():
            lstm_cell = tf.contrib.rnn.LSTMCell(config.lstm_size, state_is_tuple=True)
            if config.keep_prob < 1.0:
                lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=config.keep_prob)
            return lstm_cell

        if config.num_layers > 1:
            cell = tf.contrib.rnn.MultiRNNCell(
                [_create_one_cell() for _ in range(config.num_layers)],
                state_is_tuple=True)
        else:
            cell = _create_one_cell()

        val, _ = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32, scope="RNN")
        val = tf.transpose(val, [1,0,2])

        with tf.name_scope("Output_Layer"):
            last = tf.gather(val, int(val.get_shape()[0]) - 1, name="Last_LSTM_Outputs")

            weight = tf.Variable(tf.truncated_normal([config.lstm_size, config.input_size]), name="weights")
            bias = tf.Variable(tf.constant(0.1, shape=[config.input_size]), name="biases")
            prediction = tf.matmul(last, weight) + bias

            tf.summary.histogram("Last_LSTM_Outputs", last)
            tf.summary.histogram("weights", weight)
            tf.summary.histogram("biases", bias)

        with tf.name_scope("Train_Attrs"):
            loss = tf.reduce_mean(tf.square(prediction-targets), name="MSE_Loss")
            optimizer = tf.train.AdamOptimizer(learning_rate)
            minimize = optimizer.minimize(loss, name="loss_adam_minimize")
            tf.summary.scalar("MSE_Loss", loss)

        for op in [prediction, loss]:
            tf.add_to_collection("Ops_to_Restore", op)
    
    return lstm_graph

### Computing Learning Rates

In [ ]:
def _compute_learning_rates():
    learning_rates_to_use = [config.init_learning_rate * (config.learning_rate_decay ** max(float(i + 1 - config.init_epoch), 0.0)) for i in range(config.max_epoch)]
    print("Middle learning rate:", learning_rates_to_use[len(learning_rates_to_use) // 2])
    return learning_rates_to_use

### Training LSTM Model

In [ ]:
def train_lstm_graph(lstm_graph, stock_name="GSPC.csv", norm=True):
    
    stock_dataset = StockDataSet(stock_name, config.input_size, num_steps=config.num_steps, normalized=norm)
    
    final_prediction = []
    final_loss = None
    
    graph_name = "{sn}_lr={lr}_lr-decay={lrd}_lstm-size={size}_num-steps={steps}_input-size={inp}_batch-size={bs}_epochs={ep}"
    graph_name = graph_name.format(sn=stock_name[:stock_name.find('.')],
                     lr=config.init_learning_rate,
                     lrd=config.learning_rate_decay,
                     size=config.lstm_size,
                     steps=config.num_steps,
                     inp=config.input_size,
                     bs=config.batch_size,
                     ep=config.max_epoch)
    
    print(graph_name)
    
    learning_rates_to_use = _compute_learning_rates()
    #print('learning rates:\n', learning_rates_to_use)
    with tf.Session(graph=lstm_graph) as sess:
        
        merged_summary = tf.summary.merge_all()
        print('merged_summary:',type(merged_summary))
        writer = tf.summary.FileWriter('logs/' + graph_name)
        writer.add_graph(sess.graph)

        graph = tf.get_default_graph()
        tf.global_variables_initializer().run()

        inputs = graph.get_tensor_by_name('Inputs:0')
        targets = graph.get_tensor_by_name('Targets:0')
        learning_rate = graph.get_tensor_by_name('learn_rate:0')

        test_data_feed = {
            inputs: stock_dataset.test_X,
            targets: stock_dataset.test_y,
            learning_rate: 0.0
        }

        loss = graph.get_tensor_by_name('Train_Attrs/MSE_Loss:0')
        minimize = graph.get_operation_by_name('Train_Attrs/loss_adam_minimize')
        prediction = graph.get_tensor_by_name('Output_Layer/add:0')

        for epoch_step in range(config.max_epoch):
            current_lr = learning_rates_to_use[epoch_step]

            for batch_X, batch_y in stock_dataset.generate_one_epoch(config.batch_size):
                train_data_feed = {
                    inputs: batch_X,
                    targets: batch_y,
                    learning_rate: current_lr
                }
                train_loss, _ = sess.run([loss, minimize], train_data_feed)

            if epoch_step % 10 == 0:
                test_loss, _pred, _summary = sess.run([loss, prediction, merged_summary], test_data_feed)
                assert len(_pred) == len(stock_dataset.test_y)
                print_out = "Epoch {epoch_step} lr={lr} test_loss={test_loss}".format(epoch_step=epoch_step,
                                                                                     lr=current_lr,
                                                                                     test_loss=test_loss)
                print(print_out)
                if epoch_step % 50 == 0:
                    print("Predictions:", [(
                        map(lambda x: round(x, 4), _pred[-j]),
                        map(lambda x: round(x, 4), stock_dataset.test_y[-j])
                    ) for j in range(5)])

            writer.add_summary(_summary, global_step=epoch_step)

        print("Final Results:")
        final_prediction, final_loss = sess.run([prediction, loss], test_data_feed)
        print(final_prediction, final_loss)

        graph_saver_dir = os.path.join('models', graph_name)
        if not os.path.exists(graph_saver_dir):
            os.mkdir(graph_saver_dir)

        saver = tf.train.Saver()
        saver.save(sess, os.path.join(graph_saver_dir, "stock_rnn_model_{gn}.ckpt".format(gn=graph_name)),
                                      global_step=epoch_step)

    with open("final_predictions.{}.json".format(graph_name), 'w') as fout:
        fout.write(json.dumps(final_prediction.tolist()))
    
    return graph_name, stock_dataset
        

# Running the Model

In [ ]:
lstm_graph = create_lstm_graph()

In [ ]:
trained_graph, stock_dataset = train_lstm_graph(lstm_graph, norm=True)

# Restoring Model and Running

In [ ]:
import os
import tensorflow as tf

In [ ]:
def prediction_of_model(graph_name, max_epoch, test_X, test_y):
    test_prediction = None
    test_loss = None
    
    with tf.Session() as sess:
        graph_path = os.path.join('models',graph_name,'stock_rnn_model_{graph}.ckpt-{epochs}.meta'.format(
            graph=graph_name, epochs=max_epoch-1))
        checkpoint_path = os.path.join('models', graph_name)
        
        importer = tf.train.import_meta_graph(graph_path)
        importer.restore(sess, tf.train.latest_checkpoint(checkpoint_path))
        
        graph = tf.get_default_graph()
        

        inputs = graph.get_tensor_by_name('Inputs:0')
        targets = graph.get_tensor_by_name('Targets:0')
        learning_rate = graph.get_tensor_by_name('learn_rate:0')

        test_data = {
            inputs: test_X,
            targets: test_y,
            learning_rate: 0.0
        }
        
        loss = graph.get_tensor_by_name('Train_Attrs/MSE_Loss:0')
        prediction = graph.get_tensor_by_name('Output_Layer/add:0')
        
        test_prediction, test_loss = sess.run([prediction, loss], test_data)
        
    return test_prediction, test_loss

In [ ]:
X = np.concatenate((stock_dataset.train_X, stock_dataset.test_X), axis=0)
Y = np.concatenate((stock_dataset.train_y, stock_dataset.test_y), axis=0)

In [ ]:
X.shape, Y.shape

In [ ]:
res, loss = prediction_of_model(trained_graph, config.max_epoch, X, Y)

In [ ]:
print(loss)

# Visualization

In [ ]:
res.shape

In [ ]:
result_size = res.shape[0]*res.shape[1]
res_1d = np.ravel(res)[int(result_size/result_size+1):]
Y_1d = np.ravel(Y)[int(result_size/result_size+1):]
X_1d = np.array(list(range(Y_1d.shape[0])))

In [ ]:
plt.plot(X_1d, Y_1d,c='r', label='Actual Data')
plt.plot(X_1d, res_1d,c='b', label='Predictions')
plt.ylim(-Y_1d.max()*0.8, Y_1d.max())
plt.savefig(trained_graph+'.jpg')
plt.show()

<br><br><br><br><br><br><br><br><br><br><br><br>